# King County Home Improvements
![Hero Lake Washington, King County](images/hero-lake-washington-xlg.jpg)
<br>

**Author**: Carl Schneck <br>
**Program**: Data Science Flex <br>
**Phase 2 Project**

---

In [ ]:
# Initializations
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import stats
import data_preparation_functions as dp
import figure_functions as fg

%matplotlib inline

## Overview

This project analyzes King County housing sales data in order to help lead a wholesale real estate investor make educated decisions on which home improvements best improve sales prices in the area. King County is the most populous county in Washington State, and 13th in the country.  

---

## Business Understanding

A wholesale real estate investor wants to get a better idea of which improvements relate to the the biggest increase in sales price. By finding out this information they can better access which projects are more worthwile and lead to the largest profit margin. Through a linear regression analysis we can figure out which features have the largest affect on sales price by looking at our models coefficients. 

The area of interest is King County, Washington. Which is the most populous county of Washington state, and ranked 13th in the country. In order to complete the objective a dataset containing sales data for King County spanning the years 2014 to 2015 was analyzed. 

---

## Data Understanding

Our dataset includes information on houses sold in the timeframe spanning the years 2014 through 2015 and 70 zipcodes of King County. There are a total of 21,597 entries with 21 columns worth of information. For this analysis we will cut down this data to features we believe will be helpful for the client. These include features that can be improved after the purchase of a home. Features dealing with location, view or neighbooring properties are things that in most cases are impossible to change so will be ommited.    

After taking the above into consideration we began modelling with the features below, with `price` being our target variable. For a more in depth analysis of why the other features were not included, as well as the the preperation and modeling process, please observe the EDA and Modeling notebook linked <a href="EDA_and_Modelling.ipynb">here</a>.

- `price` - Sales price of house
- `bedrooms` - Number of bedrooms
- `bathrooms` - Number of bathrooms
- `sqft_living` - Square footage of living space
- `floors` - Number of floors
- `condition` - Overall maintanence condition of the house
    - 1 = Poor
    - 2 = Fair
    - 3 = Average
    - 4 = Good
    - 5 = Very Good
- `grade` - Overall construction and design grade of the house
    - 3 = Poor
    - 4 = Low
    - 5 = Fair
    - 6 = Low Average
    - 7 = Average
    - 8 = Good
    - 9 = Better
    - 10 = Very Good
    - 11 = Excellent
    - 12 = Luxury
    - 13 = Mansion
    
---

## Data Preparation

There were a few steps taken to prepare the data. 

1. Drop the unnecessary columns
2. Convert the categorical columns to numerical columns
    - (`grade`, `condition`)
3. Drop any duplicates
4. Drop any outliers

The initial dataframe after preparation is shown below. 

In [ ]:
# Loads initial prepared dataframe from the EDA_and_Modelling notebook
init_df = pd.read_pickle('data/init_df.pk1')
init_df.head(3)

The data was then split into a training and test set with a ratio of 4:1 in order to have a dataset to validate our model.

---

## Modeling

In order to solve our clients problem we need to be able to infer information from our model, therefore the inner mechanisms have to be kept simple and understandable. One type of model that can fit this description is a Linear Regression Model. In theory once the model is complete we should be able to pull out the coefficients related to each feature, and have a good approximation on their relationships with the target variable. Thus this type of model was chosen for this project.

After much trial and error the final model only contained three features.

- `sqft_living`
- `grade`
- `condition`

This was mostly due to the fact that some features had high correlation to each other causing the models to perform poorly. These included the relationships of `bedrooms` and `bathrooms` with `sqft_living`. This makes sense considering they are part of the living space and adding to the square footage. The feature `floors` was later taken out due to it's high p-value, thus providing a lack of confidence in it's results.

The model also had to be limited to homes with a sales price under 1.5 million dollars. This was performed because the `price` variable had a long right tail, causing the model to lack normality of its residuals.

Scaling was also performed on the model since the variable `sqft_living` contained much higher values compared to the other two features, possibly causing a bias in the results.

The final model can be previewd below

---

In [ ]:
# Loads final model sets
X_train = pd.read_pickle('data/X_train_final.pk1')
y_train = pd.read_pickle('data/y_train_final.pk1')

X_test = pd.read_pickle('data/X_test_final.pk1')
y_test = pd.read_pickle('data/y_test_final.pk1')

In [ ]:
# Creates and previews final model
model = dp.model_summary(X_train, y_train)
model.summary()

In [ ]:
model.params.sort_values(ascending=False)

The top two features from this model are `sqft_living` and `grade`, thus will be the recommended features to improve to reach the greatest profit. 

Since the model was scaled these coefficients cannot be directly used. The independant features of the model were scaled using the formula below.

<br>
<center>$$x_{iscaled} = \displaystyle \frac{x_{i} - x_{min}}{x_{max} - x_{min}}$$</center>
<br>

- $x_{iscaled}$ is the scaled value of $x_{i}$
- $x_{i}$ are the individual values in feature x
- $x_{min}$ is the minimun value in feature x
- $x_{max}$ is the maximum value in feature x

What's important for our analysis is the rate of change. Since the transformation was linear we only need to take the denominator into consideration. Thus the rate of change per unit of the feature can be found by using the equation below, where c is the coeffiecient of the feature calculated by the model.

<br>
<center>$$rate = \displaystyle \frac{c}{x_{max} - x_{min}}$$</center>
<br>

The rate of change per unit has been saved in the dataseries below.

In [ ]:
# Loads rates of change per unit from pickle file
rates = pd.read_pickle('data/rates.pk1')

In [ ]:
# Figures showing change in dollars for change in sqft_living and grade
fg.top_rate_plots(rates)

The figures above show the linear change in housing price by increasing change in `sqft_living` and `grade`.

In [ ]:
print('The change in house price per increase in living space square footage is ${}'
      .format(round(rates.sqft_living,2)))
print('The change in house price per increase in grade is ${}'
      .format(round(rates.grade,2)))

---

## Evaluation

This model passes the linearity and multicollinearity assumptions. Though it performed poorly on the normality and homoscedastic checks. Though it did perform as well as we hoped it was the best model tested. It was determined that in order to better the model complicated procedures must be taken that would make the model hard to understand.

The model explains around 52.1% of our target variables variance with an R<sup>2</sup> score of .521. The models test and training sets both had very similar Root Mean Square Percent Errors (RMSPE) close to 41% thus proving that the model isn't over fitted. This large of an error is something that we would definently like to improve on in the future. The exact error values are shown below.

In [ ]:
train_rmspe = 100 * dp.RMSPE(X_train, y_train, model)
test_rmspe = 100 * dp.RMSPE(X_test, y_test, model)

print('Training split RMSPE: {}%'.format(round(train_rmspe, 2)))
print('Test split RMSPE: {}%'.format(round(test_rmspe, 2)))

## Conclusion

This models goal was to find the features that had the largest affect on changing the price of a home. The approach was to only include variables that were deemed changeable, then build our model using these features. The final model has a Root Mean Squared Percent Error (RMSPE) of around 41% and an R<sup>2</sup> score of .521 therefore explaining around 52% of our target variables variance. The top two features were determined to be `sqft_living` and `grade`. A change of one `sqft_living` unit equates to a \$104.63 increase in price. While a single improvement in `grade` equates to a change of \$96,872 in price. Therefore it is suggested to either increase the square footage of the living space, or to remodel the living space to increase the grade of the home. Though before taking any action an analysis of material and labor costs should be taken into consideration. Since this model had a high RMSPE and didn't pass all the assumptions, it is hard to recommend its use with full confidence.

---

## Future Improvements

This model didn't perform too well on all of the assumptions for a linear regression model. The accuracy of only 41% is also too high to make a confident prediction. Therefore some improvements may be made if the following is performed.

#### Include more Features

The initial approach to this model was to strip it of all the variables that were deemed unchangeable. This included location and land descriptions. Though these variables couldn't be picked for home improvements, they could have helped narrow the model coefficients down and improved accuracy.

#### Make multiple models for different feature categories

There are possibly some features that play a large part in the difference in price, for example `waterfront` or `zipcode`. It may be beneficial to split the data into seperate models by features like these to determine if different features play a larger part in these categories.

#### Further limit the range of the model

Looking at the homoscedastic check, the residuals on the tail with houses of higher sale value are larger than those of lower values causing it to be heteroscedastic. Thus we could possibly limit the range of the model even further and make two different models for different ranges.

#### Try the box-cox method

The target variable `price` and feature `sqft_living` appear to have logarithmic curves. While this analysis attempted to transform the data using a log function it didn't pass the assumption test for homoscedasticity. The plot of residuals over the range showed a variance that was trending upwards. Upon research this could possibly be fixed using the box-cox method. The downside being it is not recommended if the goal is to infer information from the the individual feature coefficients since the transformation is exponential. 

---